<h1 style="font-size: 45px; font-weight: bold;">Case Study</h1>

<hr style="height:18px;border-radius:5px;border-width:0;color:black;background: linear-gradient(to right, #71c4ef , #00668c, #b6ccd8)">

In [1]:
def all_date_processing():
    #ạo ra một đối tượng DataFrame của Pandas có tên là all_date với một cột duy nhất có tên là all_date. Cột chứa tất cả các ngày trong năm 2023
    all_date = pd.DataFrame({'all_date':pd.date_range('2023/01/01','2023/12/31')})
    #tạo ra một Series của Pandas có tên là weekend_cond_. Đối tượng này chứa các giá trị TRue,false xem phải là này cuối tuần hay không.
    weekend_cond_ = (all_date['all_date'].dt.dayofweek != 5) \
                & (all_date['all_date'].dt.dayofweek != 6)
    #loc df all_date theo điều kiện bên trên
    all_date = all_date .loc[weekend_cond_, :].copy()
    # tạo thêm cột month và week
    all_date['month'] = all_date['all_date'].dt.month
    all_date['week'] = all_date['all_date'].dt.isocalendar().week
    #tạo dataframe mới
    ref_date = pd.DataFrame()
    for _ in range(1 , 13):
        cond_ = all_date['month'] == _

        each_month = all_date.loc[cond_, :].copy()
        each_month['week_number'] = each_month['week'] - (each_month['week'].min() - 1)

        ref_date = pd.concat([ref_date, each_month])
    return ref_date

In [2]:
def rename_col(df):
    new_cols = []
    for i in df.columns.tolist():
        new_cols += [re.sub('[ -]', '_', i).strip().lower()]
    df.columns = new_cols

<hr style="height:8px;border-radius:5px;border-width:0;color:black;background: linear-gradient(to right, #71c4ef , #00668c, #b6ccd8)">

In [3]:
def all_members_processing():
    data_path = 'C:\\Users\\duyph\\Documents\\DAP2308R1\\final\\'
    all_mem_df = pd.read_excel(data_path + 'all_members.xlsx')
    #rename col
    rename_col(all_mem_df)
    #fillna cho cột in_sdu_from và in_sdu_to
    all_mem_df['in_sdu_from']= all_mem_df['in_sdu_from'].fillna(pd.to_datetime('2023/01/01'))
    all_mem_df['in_sdu_to'] = all_mem_df['in_sdu_to'].fillna(pd.to_datetime('2023/12/31'))
    #gán cho cột in_sdu_from và in_sdu_to vào 2 biến start_day và end_day 
    start_day = all_mem_df['in_sdu_from']
    end_day = all_mem_df['in_sdu_to']
    #tạo thêm cột sdu_full_date với giá trị là danh sách ngày được tạo bằng cách tạo biến lặp _ trong khoảng ngày được tạo ra 
    # bằng hàm date_range từ start_day đến end_day tại i trong khoảng số dòng của bảng all_mem_df
    all_mem_df['sdu_full_date'] = [[_ for _ in pd.date_range(start_day[i], end_day[i])] for i in range(all_mem_df.shape[0])]
    #lọc lấy 2 cột acc và sdu_full_date
    all_mem_sdu = all_mem_df.loc[:,['acc', 'sdu_full_date' ]].copy()
    #tách trải giá trị trong cột sdu_full_date chứa các danh sách thành các hàng riêng lẻ.
    all_mem_sdu = all_mem_sdu.explode('sdu_full_date')
    all_mem_off = all_mem_df.loc[all_mem_df['off_from'].isnull() == False, :].copy()
    start_day_off = all_mem_off['off_from'].tolist()
    end_day_off = all_mem_off['off_to'].tolist()
    all_mem_off['off_full_date'] = [[_ for _ in pd.date_range(start_day_off[i], end_day_off[i])] for i in range(all_mem_off.shape[0])]  
    all_mem_off = all_mem_off.loc[:,['acc', 'off_full_date' ]].copy()
    all_mem_off = all_mem_off.explode('off_full_date')
    return all_mem_df, all_mem_sdu, all_mem_off
def work_days_handle():
    all_mem_df, all_mem_sdu, all_mem_off= all_members_processing()
    ref_date = all_date_processing()
    work_days = pd.merge(all_mem_sdu , all_mem_off
                     ,how = 'left'
                     ,left_on = ['acc', 'sdu_full_date']
                     ,right_on = ['acc', 'off_full_date']) 
    work_days = work_days.loc[work_days['off_full_date'].isnull(),['acc', 'sdu_full_date']].copy()
    work_days.rename(columns = {'sdu_full_date':'working_day'}, inplace = True)
    work_days.reset_index(drop = True, inplace = True)
    work_days = pd.merge(work_days , ref_date
                     ,how = 'inner'
                     ,left_on = ['working_day']
                     ,right_on = ['all_date']
                    )
    return work_days
def mem_by_week_handle():
    work_days = work_days_handle()
    #nhóm các hàng trong DataFrame work_days theo cột month và week_number, và đếm số giá trị duy nhất trong cột acc cho mỗi nhóm.
    mem_by_week = work_days.groupby(['month', 'week_number'])['acc'].nunique().reset_index()
    return mem_by_week

<hr style="height:8px;border-radius:5px;border-width:0;color:black;background: linear-gradient(to right, #71c4ef , #00668c, #b6ccd8)">

In [4]:
def allc_handle():
    data_path = 'C:\\Users\\duyph\\Documents\\DAP2308R1\\final\\'
    allc_df = pd.read_excel(data_path + 'allocation_report.xlsx')
    rename_col(allc_df)
    allc_df.rename(columns = {'hours_/_day':'hours/day'}, inplace = True)
    #Project duration handle
    allc_df = allc_df.loc[allc_df['username'].isnull() == False, :].copy().reset_index(drop=True)
    #tạo thêm cột project_duration với giá trị là danh sách ngày được tạo bằng cách tạo biến lặp _ trong khoảng ngày được tạo ra 
    # bằng hàm date_range từ from_date đến to_date tại i trong khoảng số dòng của bảng allc_df
    allc_df['project_duration'] = [[_ for _ in pd.date_range(allc_df['from_date'][i],allc_df['to_date'][i])] for i in range(allc_df.shape[0])]
    allc_df = allc_df.explode('project_duration')
    return allc_df

In [5]:
def allc_by_week_handle():
    allc_df = allc_handle()
    work_days = work_days_handle()
    #tạo df in_project_days bằn cách  nhóm các hàng trong DataFrame allc_df theo hai cột: 'username' và 'project_duration'. 
    #Sau đó, các giá trị trong cột 'hours/day' được tổng hợp lại cho mỗi nhóm 
    in_project_days = allc_df.groupby(['username', 'project_duration'])['hours/day'].sum().reset_index()
    in_project_days['hours/day'].unique()
    allc_date_df = pd.merge(work_days, in_project_days
             ,how = 'left'
             ,left_on = ['acc', 'working_day']
             ,right_on = ['username', 'project_duration'])
    #xóa ba cột khỏi DataFrame 
    allc_date_df.drop(columns = ['username', 'project_duration', 'all_date'], inplace = True)
    allc_date_df['hours/day'] = allc_date_df['hours/day'].fillna(0)
    hours_by_week = allc_date_df.groupby(['acc', 'month', 'week_number'])['hours/day'].sum().reset_index()
    days_by_week = allc_date_df.groupby(['acc', 'month', 'week_number'])['working_day'].count().reset_index()
    allc_by_week = pd.merge(hours_by_week, days_by_week
                            ,how = 'inner'
                            ,on = ['acc', 'month', 'week_number']
                           )
    allc_by_week['calendar_effort'] = allc_by_week['hours/day'] / (allc_by_week['working_day'] * 8)
    cond_list = [allc_by_week['calendar_effort'] == 0
             ,allc_by_week['calendar_effort'] == 1
            ]
    values = ['IDLE', 'Allocated']
    allc_by_week['calendar_effort_type'] = np.select(cond_list, values, 'Not full Allocated')
    return allc_date_df, hours_by_week, days_by_week, allc_by_week
def allocated_by_week_handle():
    allc_date_df, hours_by_week,  days_by_week, allc_by_week = allc_by_week_handle()
    allocated = allc_by_week.loc[allc_by_week['calendar_effort_type'] == 'Allocated',:].copy()
    allocated_by_week = allocated.groupby(['month', 'week_number'])['acc'].nunique().reset_index()
    return allocated, allocated_by_week
def not_full_allc_by_week_handle():
    allc_date_df, hours_by_week,  days_by_week, allc_by_week = allc_by_week_handle()
    not_f_allocated = allc_by_week.loc[allc_by_week['calendar_effort_type'] == 'Not full Allocated',:].copy()
    not_f_allocated_by_week = not_f_allocated.groupby(['month', 'week_number'])['acc'].nunique().reset_index()
    return not_f_allocated, not_f_allocated_by_week
def idle_by_week_handle():
    allc_date_df, hours_by_week,  days_by_week, allc_by_week = allc_by_week_handle()
    idle = allc_by_week.loc[allc_by_week['calendar_effort_type'] == 'IDLE',:].copy()
    idle_by_week = idle.groupby(['month', 'week_number'])['acc'].nunique().reset_index()
    return idle, idle_by_week
def details_by_week():
    allc_df = allc_handle()
    ref_date = all_date_processing()
    # tạo ra DataFrame là project_by_week bằng cách kết hợp hai DataFrame allc_df và ref_date theo hai cột: 'project_duration' và 'all_date'
    project_by_week = pd.merge(allc_df, ref_date
                      ,how = 'inner'
                      ,left_on = ['project_duration']
                      ,right_on = ['all_date'])
    return project_by_week

<hr style="height:8px;border-radius:5px;border-width:0;color:black;background: linear-gradient(to right, #71c4ef , #00668c, #b6ccd8)">

In [ ]:
def get_list_by_week_handle(month, week):
    project_by_week = details_by_week()
    get_list_row_cond_ = (project_by_week['month'] == month)\
                        & (project_by_week['week_number'] == week)

    get_list_by_week = project_by_week.loc[get_list_row_cond_, :].copy()
    return get_list_by_week
def allocated_list_handle(month, week):
    allocated, allocated_by_week = allocated_by_week_handle()
    get_list_by_week = get_list_by_week_handle(month, week)
    all_mem_df, all_mem_sdu,  all_mem_off= all_members_processing()
    allocated_acc_cond = (allocated.loc[(allocated['month'] == month)\
                                   & (allocated['week_number'] == week)
                                   , ['acc']])['acc'].unique()

    allocated_list = get_list_by_week.loc[get_list_by_week['username'].isin(allocated_acc_cond)
                                          ,['username'
                                            ,'working_for_customer'
                                            ,'from_date'
                                            ,'to_date'
                                            ,'job'
                                            ,'hours/day']
                                         ].reset_index(drop = True)
    allocated_list = pd.merge(allocated_list, all_mem_df
                          ,how = 'left'
                          ,left_on = ['username']
                          ,right_on = ['acc']
                         )
    allocated_list = allocated_list.loc[:
                                    , ['username'
                                       ,'name'
                                       ,'branch'
                                       ,'job_title'
                                       ,'empl_class'
                                       ,'working_for_customer'
                                       ,'from_date'
                                       ,'to_date'
                                       ,'job'
                                       ,'hours/day']
                                   ].copy()
    allocated_list_by_week = allocated_list.drop_duplicates()
    return allocated_list_by_week

def not_f_allocated_list_handle(month, week):
    not_f_allocated, not_f_allocated_by_week = not_full_allc_by_week_handle()
    get_list_by_week = get_list_by_week_handle(month, week)
    all_mem_df, all_mem_sdu,  all_mem_off= all_members_processing()
    #tạo biến điều kiền là loc bảng not_f_allocated  lọc các hàng từ DataFrame not_f_allocated trong với điều kiện month, week, và chỉ chọn cột acc.
    #Kết quả trả về là một Series chứa các giá trị trong cột acc sau đó dùng .unique() để loại bỏ các giá trị trùng lặp và trả về một mảng.
    not_full_allocated_acc_cond = (not_f_allocated.loc[(not_f_allocated['month'] == month)\
                                                       & (not_f_allocated['week_number'] == week)
                                                       , ['acc']])['acc'].unique()
    #loc df get_list_by_week tại cột username có giá trị nằm trong danh sách not_full_allocated_acc_cond, 
    #và chỉ chọn các cột username, working_for_customer, from_date, to_date, job, và hours/day
    not_f_allocated_list = get_list_by_week.loc[get_list_by_week['username'].isin(not_full_allocated_acc_cond)
                                                ,['username'
                                                    ,'working_for_customer'
                                                    ,'from_date'
                                                    ,'to_date'
                                                    ,'job'
                                                    ,'hours/day']
                                               ].reset_index(drop = True)
    not_f_allocated_list = pd.merge(not_f_allocated_list, all_mem_df
                                          ,how = 'left'
                                          ,left_on = ['username']
                                          ,right_on = ['acc']
                                         )
    not_f_allocated_list = not_f_allocated_list.loc[:
                                        , ['username'
                                           ,'name'
                                           ,'branch'
                                           ,'job_title'
                                           ,'empl_class'
                                           ,'working_for_customer'
                                           ,'from_date'
                                           ,'to_date'
                                           ,'job'
                                           ,'hours/day']
                                       ].copy()
    not_f_allocated_list_by_week = not_f_allocated_list.drop_duplicates()
    return not_f_allocated_list_by_week
def idle_list_handle(month, week):
    idle, idle_by_week = idle_by_week_handle()
    all_mem_df, all_mem_sdu,  all_mem_off= all_members_processing()
    idle_acc_cond = (idle.loc[(idle['month'] == month) & (idle['week_number'] == week), ['acc']])['acc'].unique()

    idle_list = all_mem_df.loc[all_mem_df['acc'].isin(idle_acc_cond),['acc', 
                                                                'name', 
                                                                'branch', 
                                                                'job_title', 
                                                                'empl_class']].reset_index(drop = True)
    idle_list_by_week = idle_list.drop_duplicates()
    return idle_list_by_week

In [26]:
def check_empty(r):
    if r.empty:
        return 0
    else:
        return r.values[0]
def summary_table_handle(month, week):
    #tạo dict với 4 key
    sumary_table = {'month': [], 'week_number': [], 'acc': [], 'type': ['Total Members', 'Allocated', 'Not Full Allocated', 'IDLE']}
    mem_by_week = mem_by_week_handle()
    allocated, allocated_by_week = allocated_by_week_handle()
    not_f_allocated, not_f_allocated_by_week = not_full_allc_by_week_handle()
    idle, idle_by_week = idle_by_week_handle()
    #chạy vòng lặp kiểm tra empty của series tại mỗi i tương ứng với month và week
    for i in range(4):
        for _ in range(4):
            if i == 0:
                cond = (mem_by_week['month'] == month) & (mem_by_week['week_number'] == week)
                r = mem_by_week.loc[ cond , 'acc']
                r_value = check_empty(r)
                print(month, week, cond, r, r_value)
                break
            elif i == 1:
                cond = (allocated_by_week['month'] == month) & (allocated_by_week['week_number'] == week)
                r = allocated_by_week.loc[ cond,'acc']
                r_value = check_empty(r)
                break
            elif i == 2:
                cond = (not_f_allocated_by_week['month'] == month) & (not_f_allocated_by_week['week_number'] == week)
                r = not_f_allocated_by_week.loc[ cond,'acc']
                r_value = check_empty(r)
                break
            elif i == 3:
                cond = (idle_by_week['month'] == month) & (idle_by_week['week_number'] == week)
                r = idle_by_week.loc[ cond,'acc']
                r_value = check_empty(r)
                break
        sumary_table['month'].append(month) 
        sumary_table['week_number'].append(week)
        sumary_table['acc'].append(r_value)
    sumary_table_df = pd.DataFrame(sumary_table)

    return sumary_table_df

def main_app():
    
    main_app = tk.Tk()
    main_app.title("Main")
    main_app.geometry("300x250+550+200")
    main_app.resizable(width=False, height=False)
    global month_entry
    global week_entry
    
    main_frame = tk.Frame(main_app, padx = 30, pady=15)
    lbl_main = tk.Label(main_frame, text = "Allocation",font=("", 15, "bold"))
    
    month_label = tk.Label(main_frame, text="Enter month:", font=("", 10, "bold"))
    month_entry = tk.Entry(main_frame)
    week_label = tk.Label(main_frame, text="Enter week:", font=("", 10,"bold"))
    week_entry = tk.Entry(main_frame)
    submit_button = tk.Button(main_frame, text="Submit", bd=0, bg = '#7FB3D5',
                             height = 2, width = 15, command=submit)
    main_frame.grid(row = 0, column = 0)
    lbl_main.grid(row =1, column = 1, columnspan = 2)
    month_label.grid(row = 2, column = 1)
    month_entry.grid(row = 2, column = 2)
    week_label.grid(row = 3, column = 1)
    week_entry.grid(row = 3, column = 2)
    submit_button.grid(row = 4, column = 1, columnspan = 2)
    for w in main_frame.winfo_children():
        w.grid_configure(padx = 7, pady = 10)
    main_app.mainloop()
def submit():
    month = month_entry.get()
    week = week_entry.get()
    if get_month(month) & get_week(week):
        month = int(month)
        week = int(week)
        summary_table = summary_table_handle(month, week)
        allocated_list_by_week = allocated_list_handle(month, week)
        not_f_allocated_list_by_week = not_f_allocated_list_handle(month, week)
        idle_list_by_week = idle_list_handle(month, week)
        employee_file_path = 'C:\\Users\\duyph\\Documents\\DAP2308R1\\final\\'
        with pd.ExcelWriter(employee_file_path + 'master_report.xlsx'
                        ,mode = 'a'
                        ,engine = 'openpyxl'
                        ,if_sheet_exists = 'replace') as writer:
            summary_table.to_excel(writer, sheet_name = 'Summary', index = False)
            allocated_list_by_week.to_excel(writer, sheet_name = 'Allocated')
            not_f_allocated_list_by_week.to_excel(writer, sheet_name = 'Not full Allocated')
            idle_list_by_week.to_excel(writer, sheet_name = 'IDLE')
        messagebox.showinfo("Successful!", f"Exported file with month is {month}, week is {week} successful!")

In [8]:
def get_month(month):
    if month.isdigit():
        month = int(month)
        if (month >= 1) & (month <= 12):
            return True
        else:
            messagebox.showerror("Invalid!", f"Month {month} is not valid, please enter again!")
            return False
    else: 
        messagebox.showerror("Invalid!","Month must be a number from 1 to 12, please enter again!")
        return False
    

def get_week(week):
    if week.isdigit():
        week = int(week)
        if (week >= 1) & (week <= 6):
            return True
        else:
            messagebox.showerror("Invalid!", f"Week {week} is not valid, please enter again!")
            return False
    else: 
        messagebox.showerror("Invalid!", "Week must be a number from 1 to 6, please enter again!")
        return False


<hr style="height:10px;border-radius:5px;border-width:0;color:black;background: linear-gradient(to right, #71c4ef , #00668c, #b6ccd8)">

In [25]:
if __name__ == "__main__":
    import tkinter as tk
    from tkinter import ttk
    from tkinter import messagebox
    import pandas as pd
    import numpy as np
    
    
    import openpyxl
    import re
    main_app()

5 2 0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20     True
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
50    False
51    False
52    False
53    False
54    False
55    False
56    False
57    False
58    False
dtype: boolean 20    62
Name: acc, dtype: int64 62
